In [0]:
from google.colab import drive
%cd /content/drive/My Drive/ECE 247/project/project

/content/drive/My Drive/ECE 247/project/project


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import matplotlib.pyplot as plt
# for creating validation set
from sklearn.model_selection import train_test_split
# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout, ELU
from torch.optim import Adam, SGD

In [0]:
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

In [0]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [0]:
# change dimensions
 X_train_valid = X_train_valid[:, np.newaxis, :, :]
 X_test = X_test[:, np.newaxis, :, :]
# # create labels 
 y_train_valid = y_train_valid - 769
 y_test = y_test -769

# split training and validation
X_train, X_val, y_train, y_val = train_test_split(X_train_valid, y_train_valid, test_size = 0.1)
data = []
(X_train.shape, y_train.shape), (X_val.shape, y_val.shape)

(((1903, 1, 22, 1000), (1903,)), ((212, 1, 22, 1000), (212,)))

In [0]:
# transform to torch tensor
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)

X_val = torch.from_numpy(X_val)
y_val = torch.from_numpy(y_val)


In [0]:
# process the data in batches for training
batch_size = 100
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [0]:
torch.cuda.is_available()
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')

The naive CNN above seems to overfit the data. Next let's try GRU mentioned in https://arxiv.org/pdf/1802.00308.pdf


In [0]:
class Net_lstm(nn.Module):
    def __init__(self):
        super(Net_lstm, self).__init__()

        self.branch1x1 = Sequential(
                          nn.Conv2d(in_channels=1, out_channels=12, kernel_size=1), # 1
                          nn.BatchNorm2d(12),
                          nn.ELU(inplace=True),
                          nn.Dropout(0.4)
        )

        self.branch1x3 = Sequential(
                          nn.Conv2d(in_channels=1, out_channels=12, kernel_size=1),
                          nn.BatchNorm2d(12),
                          nn.ELU(inplace=True),
                          nn.Dropout(0.4),
                          nn.Conv2d(in_channels=12, out_channels=12, kernel_size=(1,3),padding=(0, 1)), 
                          nn.BatchNorm2d(12),
                          nn.ELU(inplace=True),
                          nn.Dropout(0.4),

        )
        self.branch1x5 = Sequential(
                          nn.Conv2d(in_channels=1, out_channels=12, kernel_size=1),
                          nn.BatchNorm2d(12),
                          nn.ELU(inplace=True),
                          nn.Dropout(0.4),
                          nn.Conv2d(in_channels=12, out_channels=12, kernel_size=(1,5),padding=(0, 2)), 
                          nn.BatchNorm2d(12),
                          nn.ELU(inplace=True),
                          nn.Dropout(0.4)
        )
        self.branch_pool = Sequential(
            nn.MaxPool2d(kernel_size=(1,3), stride=(1,1),padding=(0, 1)),
            nn.Conv2d(in_channels=1, out_channels=12, kernel_size=1),
            nn.BatchNorm2d(12),
            nn.ELU(inplace=True),
            nn.Dropout(0.4)
        )
        #22 x 1000 x 48

        #maxpool2d
        self.MaxPool2d = nn.MaxPool2d(kernel_size=(1,2), stride=(1,2))
        #22 x 500 x 48



        

        """self.branch2_1x2 = Sequential(
                          nn.Conv2d(in_channels=48, out_channels=12, kernel_size=(1,2),stride=(1,2),padding=(0, 0)),
                          nn.BatchNorm2d(12),
                          nn.Dropout(0.1)
        )
        self.branch2_1x4 = Sequential(
                          nn.Conv2d(in_channels=48, out_channels=12, kernel_size=(1,4),stride=(1,2),padding=(0, 1)),
                          nn.BatchNorm2d(12),
                          nn.Dropout(0.1)
        )
        self.branch2_1x8 = Sequential(
                          nn.Conv2d(in_channels=48, out_channels=12, kernel_size=(1,8),stride=(1,2),padding=(0, 3)),
                          nn.BatchNorm2d(12),
                          nn.Dropout(0.1)
        )
        self.branch_pool2 = Sequential(
            nn.MaxPool2d(kernel_size=(1,2), stride=(1,2),padding=(0, 0)),
            nn.Conv2d(in_channels=48, out_channels=12, kernel_size=1),
            nn.BatchNorm2d(12),
            nn.Dropout(0.1)
        )#11x 250 x 48"""
        
        

        """self.branch3_2x2 = Sequential(
                          nn.Conv2d(in_channels=48, out_channels=24, kernel_size=(2,2),stride=(2,2),padding=(0, 0)),
                          nn.BatchNorm2d(24),
                          nn.Dropout(0.2)
        )
        self.branch3_2x4 = Sequential(
                          nn.Conv2d(in_channels=48, out_channels=24, kernel_size=(2,4),stride=(2,2),padding=(0, 1)),
                          nn.BatchNorm2d(24),
                          nn.Dropout(0.2)
        )
        self.branch3_2x8 = Sequential(
                          nn.Conv2d(in_channels=48, out_channels=24, kernel_size=(2,8),stride=(2,2),padding=(0, 3)),
                          nn.BatchNorm2d(24),
                          nn.Dropout(0.2)
        )
                          
        self.branch_pool3 = Sequential(
            nn.MaxPool2d(kernel_size=(2,2), stride=(2,2),padding=(0, 0)),
            nn.Conv2d(in_channels=48, out_channels=24, kernel_size=1),
            nn.BatchNorm2d(24),
            nn.Dropout(0.2)
            )"""#11x 125 x 96
        
        
        self.conv1 =  Sequential (
                          nn.Conv2d(in_channels=48, out_channels=64, kernel_size=(2,8),stride=(2,2),padding=(0, 3)),
                          nn.BatchNorm2d(64),
                          nn.ELU(inplace=True),
                          nn.MaxPool2d(kernel_size=(1,2), stride=(1,2)),
                          nn.Dropout(0.4)
        ) #11x 125 x 64
        
        self.conv2 = Sequential (
                          nn.Conv2d(in_channels=64, out_channels=96, kernel_size=(1,8), stride=(1,1)),
                          nn.BatchNorm2d(96),
                          nn.ELU(inplace=True),
                          nn.MaxPool2d(kernel_size=(1,6), stride=(1,2)),
                          nn.Dropout(0.4)
        )#11x 57 x 96

        self.MaxPool2d2 = nn.MaxPool2d(kernel_size=(1,3), stride=(1,2))
        #11x 62 x 96


        self.conv4 = Sequential (
                          nn.Conv2d(in_channels=96, out_channels=128, kernel_size=(2,4), stride=(1,2)),
                          nn.BatchNorm2d(128),
                          nn.ELU(inplace=True), 
                          nn.MaxPool2d(kernel_size=(1,3), stride=(1,2)),
                          nn.Dropout(0.4)
        )#10 x 13 x 128
       
        
        self.conv5 = Sequential(
                          nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,4), stride=(1,3)),
                          nn.BatchNorm2d(256),
                          nn.ELU(inplace=True),
                          nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)),
                          nn.Dropout(0.4)
        )
        #4 x 2 x 256
        self.flatten = nn.Flatten()
        self.lstm1 = nn.LSTM(256*8, 128, 1, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(256, 64, 1, batch_first=True, bidirectional=True)
        self.lstm3 = nn.LSTM(128, 32, 1, batch_first=True, bidirectional=True)
       # self.lstm4 = nn.LSTM(64, 32, 1, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(in_features=64,  out_features=4)
        self.dropout = nn.Dropout(0.4)
        self.softmax = nn.Softmax()
    def forward(self, X):
        x_b11 = self.branch1x1(X)
        #print(x_b1.shape)
        x_b12 = self.branch1x3(X)
        #print(x_b2.shape)
        x_b13 = self.branch1x5(X)
        #print(x_b3.shape)
        x_b14 = self.branch_pool(X)
        x = torch.cat([x_b11,x_b12,x_b13, x_b14],dim=1)
        #print(x.shape)
        #x = self.conv1(x)
       # print(x.shape)
        x = self.MaxPool2d(x) 

        #x_b21 = self.branch2_1x2(x)
        #print(x_b21.shape)
        #x_b22 = self.branch2_1x4(x)
        #print(x_b22.shape)
       # x_b23 = self.branch2_1x8(x)
       # print(x_b23.shape)
        #x_b24 = self.branch_pool2(x)
        #print(x_b24.shape)
       # x = torch.cat([x_b21,x_b22,x_b23, x_b24],dim=1)
       # print(x.shape)

        #x_b31 = self.branch3_2x2(x)
        #print(x_b1.shape)
        #x_b32 = self.branch3_2x4(x)
        #print(x_b2.shape)
        #x_b33 = self.branch3_2x8(x)
        #print(x_b3.shape)
        #x_b34 = self.branch_pool3(x)
        #x = torch.cat([x_b31,x_b32,x_b33, x_b34],dim=1)
        #print(x.shape)

        x = self.conv1(x)
        #print(x.shape)
        x = self.conv2(x)
        #print(x.shape)
        #x = self.MaxPool2d2(x)
        #print(x.shape)

        x = self.conv4(x)
        #print(x.shape)
        x = self.conv5(x)
        #print(x.shape)
        x = x.permute(0,3,1,2) # important! -- without <50% 128x2x12->12x128x2
        #print(x.shape)
        x = x.reshape([-1,1,256*8])
        #print(x.shape)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        #x, _ = self.lstm4(x)
        x = x.squeeze()
        x = self.fc(x)
        return x


In [0]:
# defining the model
model = Net_lstm()
model = model.float()
# # defining the optimizer
optimizer = Adam(model.parameters(), lr=0.001)
# # defining the loss function
criterion = nn.CrossEntropyLoss()
# # checking if GPU is available
if torch.cuda.is_available():
     model = model.cuda()
     criterion = criterion.cuda()

print(model)

Net_lstm(
  (branch1x1): Sequential(
    (0): Conv2d(1, 12, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0, inplace=True)
    (3): Dropout(p=0.4, inplace=False)
  )
  (branch1x3): Sequential(
    (0): Conv2d(1, 12, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0, inplace=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Conv2d(12, 12, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
    (5): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ELU(alpha=1.0, inplace=True)
    (7): Dropout(p=0.4, inplace=False)
  )
  (branch1x5): Sequential(
    (0): Conv2d(1, 12, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0, inplace=True)
    (3): Dropout(p=0.4, inp

In [0]:

n_epochs = 30
batch_size = 64
n_batches = len(y_train)//batch_size
correct = 0
total = 0
for epoch in range(n_epochs):
    running_loss = 0
    print_every = n_batches // 10
    #start_time = time.time()
    total_train_loss = 0
    train_total = 0
    train_correct = 0
    for i,data in enumerate(train_loader, 0):
        # extract data in this batch
        inputs, labels = data

        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
        inputs = inputs.to(device)
        labels = labels.to(device)
        #Set the parameter gradients to zero
        optimizer.zero_grad()
        
        #Forward pass, backward pass, optimize
      
              
        outputs = model(inputs.float())
        loss_train = criterion(outputs.cuda(), labels.long().cuda())
        loss_train.backward()
        optimizer.step()

        #Print statistics
        running_loss += loss_train
        total_train_loss += loss_train
        

        _, train_predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
        train_total += labels.size(0)                    # Increment the total count
        train_correct += (train_predicted == labels).sum()     # Increment the correct count

        #Print every 10th batch of an epoch
       # if (i + 1) % (print_every + 1) == 0:
           # print("Epoch {}, {:d}% \t train_loss: {:.2f} ".format(
                    #epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every))
            #Reset running loss and time
            #running_loss = 0.0
        print("Training Acc = {:.2f}".format(100 * train_correct / train_total))

    #At the end of the epoch, do a pass on the validation set

    X_val = X_val.to(device)
    y_val = y_val.to(device)
    output_val = model(X_val.cuda().float())
    loss_val = criterion(output_val, y_val.long().cuda())

    _, predicted = torch.max(output_val.data, 1)  # Choose the best class from the output: The class with the best score
    total += y_val.size(0)                    # Increment the total count
    correct += (predicted == y_val).sum()     # Increment the correct count
    print("Validation Acc = {:.2f}".format(100 * correct / total))
    print("Validation loss = {:.2f}".format(loss_val))

Training Acc = 23.00
Training Acc = 21.00
Training Acc = 22.00
Training Acc = 22.00
Training Acc = 23.00
Training Acc = 24.00
Training Acc = 24.00
Training Acc = 24.00
Training Acc = 24.00
Training Acc = 25.00
Training Acc = 25.00
Training Acc = 25.00
Training Acc = 25.00
Training Acc = 25.00
Training Acc = 25.00
Training Acc = 26.00
Training Acc = 25.00
Training Acc = 25.00
Training Acc = 25.00
Validation Acc = 26.00
Validation loss = 1.39
Training Acc = 27.00
Training Acc = 30.00
Training Acc = 28.00
Training Acc = 29.00
Training Acc = 28.00
Training Acc = 28.00
Training Acc = 28.00
Training Acc = 30.00
Training Acc = 30.00
Training Acc = 30.00
Training Acc = 30.00
Training Acc = 30.00
Training Acc = 30.00
Training Acc = 30.00
Training Acc = 30.00
Training Acc = 30.00
Training Acc = 30.00
Training Acc = 30.00
Training Acc = 30.00
Validation Acc = 28.00
Validation loss = 1.38
Training Acc = 28.00
Training Acc = 30.00
Training Acc = 29.00
Training Acc = 30.00
Training Acc = 31.00
Train